In [ ]:
# !pip install sentencepiece
# !pip install torch
# !pip install transformers

In [10]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

### Experimentation with FLAN T5 (may use this and VADER)

In [2]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

C:\Users\blkeu\anaconda3\envs\CS172B311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

C:\Users\blkeu\anaconda3\envs\CS172B311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\blkeu\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

C:\Users\blkeu\anaconda3\envs\CS172B311\Lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie alte sind Sie?</s>


In [9]:
classifier = pipeline(task="sentiment-analysis")
preds = classifier("Apple, OpenAI restart talks to add generative AI to next iPhone: Bloomberg")
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'score': 0.9477, 'label': 'NEGATIVE'}]

# Fetch article titles

In [27]:
# def fetch_and_parse_xml(url):
#     # Fetch the content from URL
#     response = requests.get(url)
#     response.raise_for_status()  # This will raise an error if the fetch fails

#     # Parse XML
#     root = ET.fromstring(response.content)
#     return root

# def extract_information(root):
#     # Define the namespace mapping
#     namespaces = {
#         'sa': 'https://seekingalpha.com/api/1.0',
#         'media': 'http://search.yahoo.com/mrss/'
#     }

#     # Loop through each item in the XML
#     articles = []
#     for item in root.findall('.//channel/item', namespaces):
#         # Extract information from each item
#         title = item.find('title').text
#         link = item.find('link').text
#         pub_date = item.find('pubDate').text
#         author_name = item.find('sa:author_name', namespaces).text
        
#         # Collect article information in a dictionary
#         article_info = {
#             'title': title,
#             'link': link,
#             'published_date': pub_date,
#             'author_name': author_name
#         }
#         articles.append(article_info)

#     return articles

In [28]:
# # URL of the RSS feed
# url = "https://seekingalpha.com/api/sa/combined/AAPL.xml"

# # Process the URL
# root = fetch_and_parse_xml(url)

# # Extract and print article information
# articles = extract_information(root)
# for article in articles:
#     print(article)

In [17]:
# for article in articles:
#     titles.append(article['title'])
#     # print(article['title'])

In [13]:
def load_api_key(filepath):
    """Load and return the API key from a file."""
    with open(filepath, 'r') as file:
        api_key = file.read().strip()  # .strip() removes any leading/trailing whitespace
    return api_key

def fetch_news(api_key, query):
    url = f'https://newsapi.org/v2/everything?q=apple&apiKey='
    url += api_key
    response = requests.get(url)
    return response.json()

In [38]:
def fetch_news(api_key, query, from_date, to_date):
    """Fetch news articles from the News API based on a query and date range."""
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': query,
        'from': from_date,
        'to': to_date,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Sort by publication date
        'language': 'en'  # Assuming you want articles in English
    }
    response = requests.get(url, params=params)
    return response.json()

In [39]:
api_key = load_api_key('api_key.txt')
news_articles = fetch_news(api_key, 'Apple', '2024-04-07', '2024-05-06')
print(news_articles)

{'status': 'ok', 'totalResults': 23423, 'articles': [{'source': {'id': None, 'name': 'Pausaparafeminices.com'}, 'author': 'Bruna Tavares', 'title': 'Finest On the web Position Web sites Philippines Best Filipino Ports Casinos', 'description': 'Posts Cellular Slots Range Out of All Developers Advantages of Playing 100 percent free Slots On the internet: Greatest Real cash Mobile Harbors For people Professionals You’ve got the capability to put dollars on one approach, plus withdraw playing with anot…', 'url': 'https://www.pausaparafeminices.com/ppf/finest-on-the-web-position-web-sites-philippines-best-filipino-ports-casinos/', 'urlToImage': None, 'publishedAt': '2024-05-06T23:56:49Z', 'content': 'You’ve got the capability to put dollars on one approach, plus withdraw playing with another to possess an instant and you can pain-free payout. Since the during the Ports away from Las vegas, your o… [+5253 chars]'}, {'source': {'id': None, 'name': 'KCCI Des Moines'}, 'author': 'Beau Bowman', 

In [40]:
titles = []

for article in news_items['articles']:
    if article['title'] != "[Removed]":
        titles.append(article['title'])
        # print(article['title'])

In [41]:
titles

["Apple: Even A 'Historic' Buyback Program Isn't Going To Move The Needle",
 'Wall Street Lunch: New iPads Are Here!',
 'Apple unveils new iPad Air, iPad Pro, AI-focused M4 chip',
 'Apple: The Real Reasons Warren Buffett Is Selling',
 'Apple’s $110B buyback is larger than the market caps of 418 S&P 500 stocks',
 'Latest Apple event is on tap, along with AI chip rumors',
 'Apple: Time For A Rebrand',
 "Apple: Don't Celebrate Just Yet",
 'Apple retains top spot in global tablet market in Q1 2024',
 'Apple, Samsung hog top 10 of best-selling smartphones during Q1 2024',
 'Apple: Those Who Harp On iPhone Sales Miss The Point',
 'Apple: Stay Away During This AI Make-Or-Break Moment',
 'Wall Street Brunch: Disney Earnings, Apple Event Ahead',
 "What's next for Apple? SA analysts express skepticism about impact of massive buyback",
 'Netflix and Spotify remain teen favorites but Prime is catching up - analyst',
 'Apple results, guidance has implications for supply chain: analysts',
 "Apple: D

In [42]:
data = []
for article in news_items['articles']:
    if article['title'] != "[Removed]":
        date_only = article['publishedAt'].split('T')[0]
        data.append({'title': article['title'], 'publishedAt': date_only})

df = pd.DataFrame(data, columns=['title', 'publishedAt'])

                                                title publishedAt
0   Apple discounts MLS Season Pass to $69 for the...  2024-05-02
1   It doesn’t matter how many Vision Pro headsets...  2024-04-26
2   Apple is launching new iPads May 7: Here's wha...  2024-04-25
3   The Apple Watch Series 9 is back on sale for $299  2024-04-29
4   Apple is making progress cleaning up its dirty...  2024-04-18
..                                                ...         ...
80  Samsung shipped more phones than anyone else i...  2024-04-16
81  Apple Watch Series 9 Drops to New Low Price of...  2024-04-10
82  ¿Un Apple Pencil Pro a la vista? Esta sería la...  2024-05-06
83          Apple Event on May 7: iPads Likely - CNET  2024-04-23
84        How to Clear Your Browser Cache on MacBooks  2024-04-09

[85 rows x 2 columns]


In [49]:
df

,title,publishedAt
0,Apple discounts MLS Season Pass to $69 for the...,2024-05-02
1,It doesn’t matter how many Vision Pro headsets...,2024-04-26
2,Apple is launching new iPads May 7: Here's wha...,2024-04-25
3,The Apple Watch Series 9 is back on sale for $299,2024-04-29
4,Apple is making progress cleaning up its dirty...,2024-04-18
...,...,...
80,Samsung shipped more phones than anyone else i...,2024-04-16
81,Apple Watch Series 9 Drops to New Low Price of...,2024-04-10
82,¿Un Apple Pencil Pro a la vista? Esta sería la...,2024-05-06
83,Apple Event on May 7: iPads Likely - CNET,2024-04-23


# Sentiment Analysis

In [50]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [51]:
# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiments
def get_sentiment(text):
    return analyzer.polarity_scores(text)

# Apply sentiment analysis
df['sentiments'] = df['title'].apply(get_sentiment)
df = pd.concat([df.drop(['sentiments'], axis=1), df['sentiments'].apply(pd.Series)], axis=1)

In [52]:
df

,title,publishedAt,neg,neu,pos,compound
0,Apple discounts MLS Season Pass to $69 for the...,2024-05-02,0.000,1.000,0.000,0.0000
1,It doesn’t matter how many Vision Pro headsets...,2024-04-26,0.000,0.721,0.279,0.2732
2,Apple is launching new iPads May 7: Here's wha...,2024-04-25,0.133,0.867,0.000,-0.3182
3,The Apple Watch Series 9 is back on sale for $299,2024-04-29,0.000,1.000,0.000,0.0000
4,Apple is making progress cleaning up its dirty...,2024-04-18,0.212,0.584,0.204,-0.0258
...,...,...,...,...,...,...
80,Samsung shipped more phones than anyone else i...,2024-04-16,0.000,1.000,0.000,0.0000
81,Apple Watch Series 9 Drops to New Low Price of...,2024-04-10,0.142,0.743,0.115,-0.1027
82,¿Un Apple Pencil Pro a la vista? Esta sería la...,2024-05-06,0.000,1.000,0.000,0.0000
83,Apple Event on May 7: iPads Likely - CNET,2024-04-23,0.000,1.000,0.000,0.0000


In [47]:
data_aggregated = []

# Group by 'publishedAt' date
grouped = df.groupby('publishedAt')

for date, group in grouped:
    # Initialize sentiment sums
    pos_sum = neu_sum = neg_sum = compound_sum = 0
    titles = []

    # Loop through each article in the group
    for _, article in group.iterrows():
        sentiment = get_sentiment(article['title'])
        pos_sum += sentiment['pos']
        neu_sum += sentiment['neu']
        neg_sum += sentiment['neg']
        compound_sum += sentiment['compound']
        titles.append(article['title'])

    # Calculate average sentiments for the group
    n = len(group)
    avg_pos = pos_sum / n
    avg_neu = neu_sum / n
    avg_neg = neg_sum / n
    avg_compound = compound_sum / n

    data_aggregated.append({
        "date": date,
        "article titles": titles,
        "pos": avg_pos,
        "neu": avg_neu,
        "neg": avg_neg,
        "compound": avg_compound
    })

df_aggregated = pd.DataFrame(data_aggregated, columns=['date', 'article titles', 'pos', 'neu', 'neg', 'compound'])

          date                                     article titles       pos  \
0   2024-04-07  [Fender Tone Master Pro Review: The Classiest ...  0.000000   
1   2024-04-08  [Tesla is settling with the family of the Appl...  0.126667   
2   2024-04-09  [Welcome to the Age of Technofeudalism, Apple ...  0.107667   
3   2024-04-10  [Apple Store employees in New Jersey are tryin...  0.019167   
4   2024-04-11  [Apple TV's Dark Matter series takes on one of...  0.157000   
5   2024-04-13  [Roku Breach Hits 567,000 Users, This week’s t...  0.000000   
6   2024-04-15  [Dropbox CEO Drew Houston wants you to embrace...  0.173000   
7   2024-04-16  [It's No Surprise That Samsung Phones Are Outs...  0.046444   
8   2024-04-17  [Delta Is an iOS Game Boy Emulator That (Likel...  0.000000   
9   2024-04-18  [Apple is making progress cleaning up its dirt...  0.102000   
10  2024-04-19  [Apple Blocks Several U.S.-Based Messaging App...  0.000000   
11  2024-04-20  [How to change or cancel your music 

In [53]:
df_aggregated['number of articles'] = df_aggregated['article titles'].apply(lambda x: len(x))

In [56]:
df_aggregated.to_csv('04-07-2024_05-06.2024_sentiments.csv', encoding='utf-8', index=False)